In [ ]:
# evaluate
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
import json
from pathlib import Path
import random
import yaml
import os

import pandas as pd
from datasets import Dataset


with open("configs/config.yaml", "r") as infile:
    config = yaml.load(infile, Loader=yaml.FullLoader)

os.environ["OPENAI_API_KEY"] = config["open_ai_token"]


In [ ]:

DATA_PATH = Path("../data/etl/")

with open(DATA_PATH / "raw/documents/rulesguru.json", "r", encoding="utf-8") as infile:
    data = json.load(infile)



In [ ]:

dataset = []
for level in list(set([d["question"]["level"] for d in data])):
    questions = [d for d in data if d["question"]["level"] == level]
    questions = random.choices(questions, k=5)
    dataset.extend(
        [
            {
                "question": q["question"]["questionSimple"],
                "ground_truth": q["question"]["answerSimple"],
                "level": level,
            }
            for q in questions
        ]
    )

dataset


In [ ]:
from mtg.bot import mtg_chain
from mtg.history import ChatHistory
from mtg.objects import MessageType

history = ChatHistory()

llm = mtg_chain.create_llm(model_name="gpt-4-0125-preview", temperature=1.0)



In [ ]:
# for level in list(set([d["level"] for d in dataset])):

level = "1"
for document in [d for d in dataset if d["level"] == level]:
    memory = mtg_chain.create_chat_memory(llm=llm, max_token_limit=1000)
    rules_chain = mtg_chain.create_rules_chain(llm=llm, memory=memory)

    cards = history.data_service.get_cards(document["question"], k=5)
    processed_text, matched_cards = history.replace_card_names_with_urls(
        text=document["question"], cards=cards, message_type=MessageType.RULES
    )
    rules = history.data_service.get_rules(document["question"])
    keywords = []
    for card in matched_cards:
        keywords.extend(card.keywords)
    if keywords:
        rules.extend(history.data_service.get_rules(".".join(keywords)))

    contexts = []
    for card in cards:
        contexts.append(card.to_text(include_price=False))
    for rule in rules:
        contexts.append(rule.text)
    document["contexts"] = contexts

    answer = rules_chain.invoke(
        {
            "human_input": document["question"],
            "card_data": "\n".join([card.to_text(False) for card in cards]),
            "rules_data": "\n".join([rule.text for rule in rules]),
        }
    )

    document["answer"] = answer.content


evaluation_set = [d for d in dataset if d["level"] == level]
with open(
    DATA_PATH / "processed/evaluation/rulesguru_{level}.json", "w", encoding="utf-8"
) as outfile:
    json.dump(evaluation_set, outfile)



In [ ]:

dataset = Dataset.from_pandas(pd.DataFrame(evaluation_set))
result = evaluate(
    dataset,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
    llm=llm,
)

result



In [ ]:

for key, value in result.items():
    print(f"{key}: {value:.2f}")



In [ ]:
from mtg.bot import mtg_chain
from mtg.history import ChatHistory
from mtg.objects import MessageType

history = ChatHistory()

llm = mtg_chain.create_llm(model_name="gpt-4-0125-preview", temperature=1.0)


question = "Alvaro is at 29 life and attacks with Serra Ascendant. Natalie blocks with Dragonlord Ojutai. Does either creature die? How much life does Alvaro gain?"
context = """
Rules Guru is a site where professional magic the Gathering Judges upload question answer pairs about magic the gathering.
Rules Guru Data:
question: Alvaro is at 29 life and attacks with Serra Ascendant. Natalie blocks with Dragonlord Ojutai. Does either creature die? How much life does Alvaro gain?
answer: Neither creature dies, and Alvaro gains 1 life.\n\nFirst the game determines how much damage each creature will deal. Then all combat damage is dealt simultaneously, and Alvaro gains that much life. Serra Ascendant's toughness changes at the same time. Then state-based actions are checked, Serra Ascendant will be a 6/6 and neither creature will die.

Data about Keywords: 
Lifelink\nA keyword ability that causes a player to gain life. See rule 702.15, Lifelink.
Permanents with lifelink cause their controller to gain life whenever they deal damage equal to the amount of damage dealt. Lifelink as a keyword was introduced in Future Sight, though the ability had previously existed on numerous cards, with rules errata retroactively changing these to lifelink.  Cards with similar abilities were not changed in this way.  Lifelink was a triggered ability when it was issued but is now a static ability due to the Magic 2010 rules changes.  (Cards that previously had a lifelink-like ability have been issued further errata to return them to their original functionality.  The lone exception to this is the Mirrodin card Loxodon Warhammer, which, since it was reprinted in Tenth Edition with the lifelink keyword, retains that rather than the original functionality on all editions).  Lifelink is found mostly on white cards, and also on black cards.

"""

card_data = """
Dragonlord Ojutai\ntype: Legendary Creature \u2014 Elder Dragon\ncost: {3}{W}{U}\npower/toughness: 5/4\nkeywords: Flying\ncolor identity: U W\nFlying\nDragonlord Ojutai has hexproof as long as it's untapped.\nWhenever Dragonlord Ojutai deals combat damage to a player, look at the top three cards of your library. Put one of them into your hand and the rest on the bottom of your library in any order.,
Serra Ascendant\ntype: Creature \u2014 Human Monk\ncost: {W}\npower/toughness: 1/1\ncolor identity: W\nLifelink (Damage dealt by this creature also causes you to gain that much life.)\nAs long as you have 30 or more life, Serra Ascendant gets +5/+5 and has flying.
"""

memory = mtg_chain.create_chat_memory(llm=llm, max_token_limit=1000)
rules_chain = mtg_chain.create_rules_chain(llm=llm, memory=memory)
answer = rules_chain.invoke(
    {
        "human_input": question,
        "card_data": card_data,
        "rules_data": context,
    }
)
print(answer.content)



In [ ]:
other_questions = [
    "Can you explain how the stack works in Magic: The Gathering?",
    "What happens if I cast a spell with lifelink and my opponent counters it?",
    "If I control two 'Platinum Angel' cards and lose life below zero, do I lose the game?",
    "How does 'double strike' work when combined with 'trample'?",
    "How does 'deathtouch' work when combined with 'trample'?",
    "What are the rules for assigning damage to multiple blockers?",
    "Can I activate an ability of a creature the turn it comes into play?",
    "How do 'hexproof' and 'shroud' differ in Magic: The Gathering?",
    "What happens if a card's ability triggers during my opponent's end step?",
    "If I have 'Leyline of Sanctity' in play, can my opponent target me with discard spells?",
    "Can you explain how 'indestructible' and 'destroy' effects interact?",
    "Can you explain the basics of how to play Magic?",
]

